In [1]:
import nltk
import json
import csv
import numpy as np
from collections import defaultdict
from datetime import datetime
from datetime import timedelta

In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import requests
requests.packages.urllib3.disable_warnings()

In [3]:
reviews = []
with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        reviews.append(json_line)

In [4]:
# Possibly train our own model later

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')]

training_docs = subj_docs + obj_docs

sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])


/home/amcire/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentim_analyzer = SentimentIntensityAnalyzer()


In [6]:
sentim_analyzer.polarity_scores("hasn't been a good shooter")

{'compound': -0.3412, 'neg': 0.445, 'neu': 0.555, 'pos': 0.0}

In [6]:

with open("stablebiz.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    x = list(reader)
    mat = np.array(x)
business_ids = mat[1:,1]
review_counts = [int(val) for val in mat[1:,6]]


In [7]:
valid_business_ids = []
for i in range(len(business_ids)):
    if review_counts[i] > 3*52:
        valid_business_ids.append(business_ids[i])
print(len(valid_business_ids))

234


In [8]:
ordered_review_ids = []
bid_to_rid_and_date_dict = defaultdict(list)
rid_to_date = {}
rid_to_text = {}
rid_to_bid_dict = {}
for review in reviews:
    if review["business_id"] in valid_business_ids:
        ordered_review_ids.append(review["review_id"])
        bid_to_rid_and_date_dict[review["business_id"]].append((review["review_id"], review["date"]))
        rid_to_date[review["review_id"]] = review["date"]
        rid_to_text[review["review_id"]] = review["text"]



In [9]:
for bid, lst in bid_to_rid_and_date_dict.items():
    print(bid)
    print(len(lst))
    print

3Mx4renubPRnjHUw1n2UkA
752

Hp8k_RpSIWSeJguyaQpfIw
378

3qlqzQrwh8hjBltlgFgsJQ
262

KhWl4Mwhm_Oqq0zIIz-0wQ
255

Pthe4qk5xh4n-ef-9bvMSg
323

j8pUAS0qdEcPsjkqEDNbhw
475

698nukvxe7mq2Fra5vDikQ
452

h7s4MRVvB726jWHCHiQ9kw
319

OETh78qcgDltvHULowwhJg
2700

Xg5qEQiB-7L6kGJ5F4K3bQ
1083

P7pxQFqr7yBKMMI2J51udw
2474

0EgYXYjt2XJL4hlsKnzrcw
441

OKZkEXzJt0XIamHRRTX-8g
717

kABF0hYfAEnl166mn1zR1A
361

IZivKqtHyz4-ts8KsnvMrA
865

-FtngCwHCD2tRlH8jpj_Ag
388

JmI9nslLD7KZqRr__Bg6NQ
1977

PbyqDJf815zRn-7p6vYpAQ
774

_XN-GwzZwAyIqLKJsl2htg
689

rbcfYmJtqwIkk17IeOI5Kw
497

nUpz0YiBsOK7ff9k3vUJ3A
893

n8Zqqhff-2cxzWt_nwhU2Q
818

SeNOJ2zYHziptxLuiRINLg
457

44kd3YdkhXj5XiSPs5XNjQ
449

1DP5vHxJzCDbgawv0WC3yA
604

eLFfWcdb7VkqNyTONksHiQ
1619

uuGL8diLlHfeUeFuod3F-w
902

ozMd45nXHTCl3mdOwouwlw
245

thLX_k20SPJ0KyusGTBIHw
509

yfxDa8RFOvJPQh0rNtakHA
2247

i9D9xPBV0gR1Ja9kbY4NCw
367

dn_ipqbm7_jUz5X3rDez_A
1278

5q6Xh-UcJa78bp6dzyaE7w
273

rioQ_p2pILNbJ4Xp5jW6-Q
598

cYwJA2A6I12KNkm2rtXd5g
5116

ThNGovQZjZ5m

In [10]:
start_date_str = "2016-01-01"
start_date_obj = datetime.strptime(start_date_str, '%Y-%m-%d')

end_date_str = "2017-01-20"
end_date_obj = datetime.strptime(end_date_str, '%Y-%m-%d')

bid_to_scores = {}

weeks_to_consider = 54
desired_reviews_per_week = 3

for bid, lst in bid_to_rid_and_date_dict.items():
    print(bid)
    filtered_list = [rid_date_pair for rid_date_pair in lst 
                     if rid_date_pair[1] > start_date_str]
    filtered_list = sorted(filtered_list, key=lambda x: x[1])
    print(len(filtered_list))
#     print(filtered_list)
#     break
    if len(filtered_list) < desired_reviews_per_week*weeks_to_consider:
        continue
    sentiment_scores = np.zeros(weeks_to_consider)
    curr_date_comparison = start_date_obj
    next_date_comparison = start_date_obj + timedelta(days=7)
    sum_curr_sentiment_score = 0
    c = 0
    week_ind = 0
    curr_index = 0
    while True:
        
        (rid, date) = filtered_list[curr_index]
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        
#         print(curr_date_comparison)
#         print(date_obj)
        if curr_date_comparison <= date_obj <= next_date_comparison:
            score = sentim_analyzer.polarity_scores(rid_to_text[rid])["compound"]
            sum_curr_sentiment_score += score
            c += 1
            curr_index += 1
        else:
            sentiment_scores[week_ind] = (sum_curr_sentiment_score / max(c,1))
            sum_curr_sentiment_score = 0
            c = 0
            week_ind += 1
            curr_date_comparison = next_date_comparison
            next_date_comparison = curr_date_comparison + timedelta(days=7)
        
        if curr_date_comparison >= end_date_obj:
            if sum_curr_sentiment_score > 0 and c > 0:
                sentiment_scores[week_ind] = (sum_curr_sentiment_score / max(c,1))
            break
        if curr_index >= len(filtered_list):
            break
    
    bid_to_scores[bid] = sentiment_scores
    print(sentiment_scores)
    print
        

3Mx4renubPRnjHUw1n2UkA
341
[ 0.67391429  0.5811      0.64468571  0.60675     0.80436667  0.21365
  0.23808     0.64082     0.48648     0.84525556  0.79534444  0.89703
  0.51413333  0.33612222  0.62951111  0.20924286  0.19238     0.35013333
  0.9645      0.89961429  0.2535      0.52428571  0.61918889  0.43857273
  0.42460909  0.5456625   0.16328571 -0.141       0.21765714  0.600975
  0.36216667  0.28528333 -0.02172     0.11794     0.57437143 -0.017875
  0.4383      0.8086     -0.346525   -0.11651667 -0.58556667  0.9788
  0.35005714  0.394775    0.793325    0.3363     -0.31473333  0.2233375
  0.1725      0.60027778  0.38466667 -0.6908      0.6436      0.4517    ]

Hp8k_RpSIWSeJguyaQpfIw
251
[ 0.9571      0.96545     0.96460833  0.95844     0.95088889  0.9015
  0.95252222  0.9423      0.93833333  0.9603      0.87814     0.89844
  0.87138333  0.9276      0.832325    0.89994545  0.91895     0.94064
  0.8824      0.95528     0.922       0.895325    0.85513333  0.85335
  0.95621667  0.8990333

In [11]:
len(bid_to_scores)

211

In [12]:
bid_to_score_diffs = {}
for bid, scores_list in bid_to_scores.items():
    avg = np.mean(scores_list)
    score_diffs_normalized = []
    for i in range(1, len(scores_list)):
        diff = scores_list[i] - scores_list[i-1]
        score_diffs_normalized.append(diff/avg)
    bid_to_score_diffs[bid] = score_diffs_normalized
    

In [13]:
bids = list(bid_to_score_diffs.keys())
score_mat = []
for bid in bids:
    score_mat.append(bid_to_score_diffs[bid])
score_mat = np.array(score_mat)
score_mat.shape

(211, 53)

In [14]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2)
tsne_score_mat = tsne_model.fit_transform(score_mat)

In [15]:
tsne_score_mat.shape

(211, 2)

In [16]:
# Create a trace
trace = Scatter(
    x = tsne_score_mat[:,0],
    y = tsne_score_mat[:,1],
    mode = "markers"
)

data = [trace]

py.iplot(data)

In [18]:
bid_to_up_down = {}
for bid, scores_list in bid_to_scores.items():
    
    scores_up_down = []
    for i in range(1, len(scores_list)):
        diff = scores_list[i] - scores_list[i-1]
        if diff >= 0:
            scores_up_down.append(1)
        else:
            scores_up_down.append(-1)
    bid_to_up_down[bid] = scores_up_down

In [19]:
bids = list(bid_to_up_down.keys())
score_mat = []
for bid in bids:
    score_mat.append(bid_to_up_down[bid])
score_mat = np.array(score_mat)
score_mat.shape

(211, 53)

In [20]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2)
tsne_score_mat = tsne_model.fit_transform(score_mat)

In [21]:
tsne_score_mat.shape

(211, 2)

In [22]:
# Create a trace
trace = Scatter(
    x = tsne_score_mat[:,0],
    y = tsne_score_mat[:,1],
    mode = "markers"
)

data = [trace]

py.iplot(data)